In [165]:
import math as m
import numpy as np
import os

# Path to one fo the UI-PRMD movements
movement_subject_id = 'm10_s01_e01'
# pos_path = "./dataset/UI-PRMD/correct/kinect/positions/{}_positions.txt".format(movement_subject_id)
# ang_path = "./dataset/UI-PRMD/correct/kinect/angles/{}_angles.txt".format(movement_subject_id)
pos_path = "./dataset/UI-PRMD/incorrect/kinect/positions/{}_positions_inc.txt".format(movement_subject_id)
ang_path = "./dataset/UI-PRMD/incorrect/kinect/angles/{}_angles_inc.txt".format(movement_subject_id)
pos_data = np.loadtxt(pos_path, delimiter=",")
ang_data = np.loadtxt(ang_path, delimiter=",")

pos_data.shape # (num_frames, (x, y, z)*22kps)

# Transform data
num_frames = pos_data.shape[0]
num_kp = 22
num_axes = 3
p_data = pos_data.T
p_data = p_data.reshape(num_kp, num_axes, -1)
a_data = ang_data.T
a_data = a_data.reshape(num_kp, num_axes, -1)

p = np.copy(p_data)
a = np.copy(a_data)

def Rotx(theta):
    return np.matrix([[ 1, 0           , 0           ],
                      [ 0, m.cos(theta),-m.sin(theta)],
                      [ 0, m.sin(theta), m.cos(theta)]])
  
def Roty(theta):
    return np.matrix([[ m.cos(theta), 0, m.sin(theta)],
                      [ 0           , 1, 0           ],
                      [-m.sin(theta), 0, m.cos(theta)]])
  
def Rotz(theta):
    return np.matrix([[ m.cos(theta), -m.sin(theta), 0 ],
                      [ m.sin(theta), m.cos(theta) , 0 ],
                      [ 0           , 0            , 1 ]])

def eulers_2_rot_matrix(x):
    gamma_x=x[0];beta_y=x[1];alpha_z=x[2]
    return Rotz(alpha_z)*Roty(beta_y)*Rotx(gamma_x)

# convert the data from relative coordinates to absolute coordinates
def rel2abs(p, a, num_frames):
    skel = np.zeros((num_kp, num_axes, num_frames))
    for i in range(num_frames):
        """
        1 Waist (absolute)
        2 Spine
        3 Chest
        4 Neck
        5 Head
        6 Head tip
        7 Left collar
        8 Left upper arm 
        9 Left forearm
        10 Left hand
        11 Right collar
        12 Right upper arm 
        13 Right forearm
        14 Right hand
        15 Left upper leg 
        16 Left lower leg 
        17 Left foot 
        18 Left leg toes
        19 Right upper leg 
        20 Right lower leg 
        21 Right foot
        22 Right leg toes
        """

        joint = p[:,:,i]
        joint_ang = a[:,:,i]

        # chest, neck, head
        rot = eulers_2_rot_matrix(joint_ang[0,:]*np.pi/180)
        for j in range(1, 6):
            rot = rot @ eulers_2_rot_matrix(joint_ang[j, :] * np.pi/180)
            joint[j, :] = rot @ joint[j, :] + joint[j - 1, :]                

        # left-arm
        rot = eulers_2_rot_matrix(joint_ang[2,:]*np.pi/180)
        joint[6,:] =  rot@joint[6,:] +  joint[2,:]
        for j in range(7, 10):
            rot = rot @ eulers_2_rot_matrix(joint_ang[j-1, :] * np.pi/180)
            joint[j, :] = rot @ joint[j, :] + joint[j - 1, :]           

        # right-arm
        rot = eulers_2_rot_matrix(joint_ang[2,:]*np.pi/180)
        joint[10,:] =  rot@joint[10,:] +  joint[2,:]
        for j in range(11, 14):
            rot = rot @ eulers_2_rot_matrix(joint_ang[j-1, :] * np.pi/180)
            joint[j, :] = rot @ joint[j, :] + joint[j - 1, :]  

        # left-leg
        rot = eulers_2_rot_matrix(joint_ang[0,:]*np.pi/180)
        joint[14,:] =  rot@joint[14,:] +  joint[0,:]
        for j in range(15, 18):
            rot = rot @ eulers_2_rot_matrix(joint_ang[j-1, :] * np.pi/180)
            joint[j, :] = rot @ joint[j, :] + joint[j - 1, :]  

        # right-leg
        rot = eulers_2_rot_matrix(joint_ang[0, :] * np.pi / 180)
        joint[18, :] = rot @ joint[18, :] + joint[0, :]
        for j in range(19, 22):
            rot = rot @ eulers_2_rot_matrix(joint_ang[j - 1, :] * np.pi / 180)
            joint[j, :] = rot @ joint[j, :] + joint[j - 1, :]  

        skel[:,:,i] = joint
    return skel

skel = rel2abs(p, a, num_frames)

In [166]:
# order of joint connections
J = np.array([[3, 5, 4, 2, 1, 2, 6, 7, 8, 2, 10, 11, 12, 0, 14, 15, 16, 0, 18, 19, 20],
              [2, 4, 2, 1, 0, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]])

In [167]:
# Visualization
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
%matplotlib inline

fig = plt.figure(figsize=(12,10), layout = "constrained")
ax = fig.add_subplot(111)

def get_plot(i):
    ax.cla()
    
    ax.set_title(movement_subject_id + ' 2D animation') 
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    
    joint = skel[:, :, i]
    
    for j in range(J.shape[1]):
        p1 = joint[J[0,j],:]
        p2 = joint[J[1,j],:]
        ax.plot([p1[0], p2[0]], [p1[1], p2[1]], 'o-')

    ax.set_xlim(-70, 20) 
    ax.set_ylim(0, 170)

plt.rcParams['animation.html'] = 'html5'
anim = FuncAnimation(fig, get_plot, blit=False, frames=num_frames)
plt.close()

In [168]:
anim

In [169]:
%matplotlib inline
# Visualization 3D 
fig = plt.figure(figsize=(12,10))
ax = fig.add_subplot(projection = '3d')

def get_plot(i):
    ax.cla()
    
    ax.set_title('3D plot using transformed data') 
    ax.set_xlabel('z')
    ax.set_ylabel('x')
    ax.set_zlabel('y')
    
    joint = skel[:, :, i]
    
    for j in range(J.shape[1]):
        p1 = joint[J[0,j],:]
        p2 = joint[J[1,j],:]
        ax.plot([p1[2], p2[2]], [p1[0], p2[0]], [p1[1], p2[1]], 'o-')
    
    ax.set_zlim(-25, 160) 
    ax.set_ylim(-30, 30)
    ax.set_xlim(-400,-100)

plt.rcParams['animation.html'] = 'html5'
anim3d = FuncAnimation(fig, get_plot, blit=False, frames=num_frames)
plt.close()

In [170]:
anim3d